In [ ]:
from helpers import *
from gaussfit import *

from pylab import rcParams
rcParams['figure.figsize'] = 8, 5

## Configuration

In [ ]:
# list of PMTs at 0 voltage from HV files + disconnected pmts: 290
offPMTs = [1, 111, 143, 166, 192, 230, 238, 254, 222, 302, 309, 340, 353, 290 ]
offChs = PMTid_to_channel(offPMTs)

WESTonly = [ x for x in range(1,181) ]
chWESTonly = PMTid_to_channel(WESTonly)

EASTonly = [ x for x in range(181,361) ]
chEASTonly = PMTid_to_channel(EASTonly)

# drifting PMTs --> wait for full list
driftPMTsEAST = [ 282, 288, 296, 307, 310, 319, 346, 350]
driftPMTsWEST = [ 59, 63, 81, 93, 95, 142, 148, 169, 170, 179 ]
driftChsEAST = PMTid_to_channel(driftPMTsEAST)
driftChsWEST = PMTid_to_channel(driftPMTsWEST)
print(driftChsWEST)

newPMTsWest = [ 66 ]
newDriftChs = PMTid_to_channel(newPMTsWest)

# PMTs not changed
#nochangePMTs = [310, 282, 262, 248, 90]
nochangePMTs = [90]
nochangeChs = PMTid_to_channel(nochangePMTs)

In [ ]:
srcpath = "/exp/icarus/data/users/mvicenzi/pmt-calibration/calibrationdb/"
histpath = "/exp/icarus/data/users/mvicenzi/pmt-calibration/quick/"
figpath = "/exp/icarus/data/users/mvicenzi/pmt-calibration/figs/"

# equalization
#file = srcpath + "equalization_run10263_1691610710.csv" #nominal, prefix
#file = srcpath + "equalization_run10313_1692203859.csv" #nominal 
#file = srcpath + "equalization_run10333_1692368269.csv" #+30V
#file = srcpath + "equalization_run10334_1692374665.csv" #-30V
#file = srcpath + "equalization_run10335_1692377717.csv" #-50V
#file = srcpath + "equalization_run10336_1692384276.csv" #-100V

#file = srcpath + "equalization_run10890_1697390445.csv" #nominal
#file = srcpath + "equalization_run10893_1697472021.csv" #+30V
#file = srcpath + "equalization_run10894_1697477246.csv" #-30V
#file = srcpath + "equalization_run10895_1697481155.csv" #-50V
#file = srcpath + "equalization_run10896_1697485836.csv" #-100V

#file = srcpath +"backgroundphotons_run10898_1697562799.csv"
#file = srcpath + "equalization_run10900_1697567880.csv" #+30V
#file = srcpath + "equalization_run10901_1697572018.csv" #-30V

file = getMostRecentCalibration(srcpath)

## PMT gain distribution

In [ ]:
print( "Loading data from: "+ file )
run=getRun(file)
timestamp=getTimestamp(file)
data = getDataFrame(file, offPMTs, False)

fitOutput  = fitGainsDistribution( data, BINSIZE=0.002,RMIN=0.0,RMAX=1.0, p0=(0.4, 0.01) )

plt.xlim((0.25, 0.55))
plt = makeplot(timestamp, figpath+"gain_hist_"+str(run)+".pdf", *fitOutput )
plt.title("PMT Gain Equalization", fontsize=14)
name = "run"+str(run)+"_histo_all"
#name = "figs/ICARUS_Run2_PMT_Gains_start_run9342"
plt.savefig("figs/"+name+".png", dpi=100)
#plt.savefig(name+".svg", dpi=100)
#plt.savefig(name+".pdf", dpi=100)


#data[["pmt", "q", "eq"]].to_csv(histpath+"histogram_run"+str(run)+"_"+str(timestamp)+".csv")

## Scatter plots

In [ ]:
plt.scatter(data["pmt"],data["q"],marker="o")

## latest equalization target
plt.axhline(y=0.40, color="red", linestyle='dotted')

## KNOWN HV ISSUES
drift_dataWEST = data[data['pmt'].isin(driftChsWEST)]
drift_dataEAST = data[data['pmt'].isin(driftChsEAST)]
newDriftData = data[data['pmt'].isin(newDriftChs)]
plt.scatter(drift_dataWEST["pmt"],drift_dataWEST["q"],marker="x",color="red", label="HV issues - now fixed (WEST)")
plt.scatter(drift_dataEAST["pmt"],drift_dataEAST["q"],marker="x",color="magenta", label="HV issues - now fixed (EAST)")
plt.scatter(newDriftData["pmt"],newDriftData["q"],marker="x",color="orange", label="New HV issues")

## NOT EQUALIZED
nochange_data = data[data['pmt'].isin(nochangeChs)]
plt.scatter(nochange_data['pmt'],nochange_data['q'],marker="^",color="green", label="Manual fit (PMT 90, ch 279)")

# MANUALLY EQUALIZED
# manual_data = data[data['pmt'].isin(manualChs)]
#plt.scatter(manual_data['pmt'],manual_data['q'],marker="^",color="violet", label="Manual adjustment")

#plt.xlim((180,360))
#plt.ylim((0.2,0.6))
plt.xlabel("PMT channel ID")
plt.ylabel("Gains [10^7 electrons]")
plt.legend()
plt.grid()

plt.savefig("figs/run"+str(run)+"_scatter_all.png", dpi=200)

In [ ]:
## PRINTING OUTLIERS
reference = 0.396
low_bound = reference - 9*0.0034*reference
high_bound = reference + 9*0.0034*reference

ch_high = data.loc[ data["q"] > high_bound, ["pmt", "q"]]
ch_low = data.loc[ data["q"] < low_bound, ["pmt", "q"]]

print("PMT channels above "+ str(high_bound))
pmt_high = channel_to_PMTid(ch_high["pmt"].to_numpy())
print("Ch ID: ", ch_high.pmt.to_numpy() )
print("PMT ID: ", pmt_high )
print("q: ", ch_high.q.to_numpy() )

print("PMT channels below "+ str(low_bound))
pmt_low = channel_to_PMTid(ch_low["pmt"].to_numpy())
print("Ch ID: ", ch_low.pmt.to_numpy() )
print("PMT ID: ", pmt_low )
print("q: ", ch_low.q.to_numpy() )

## Comparisons and random stuff

In [ ]:
file0 = srcpath + "equalization_run10890_1697390445.csv" #nominal
file1 = srcpath + "equalization_run10893_1697472021.csv" #+30V
file2 = srcpath + "equalization_run10894_1697477246.csv" #-30V
file3 = srcpath + "equalization_run10895_1697481155.csv" #-50V
file4 = srcpath + "equalization_run10896_1697485836.csv" #-100V

remove = offPMTs + WESTonly

#data_01 = getDataFrame(file01, remove, False)
data_0 = getDataFrame(file0, remove, False)
data_1 = getDataFrame(file1, remove, False)
data_2 = getDataFrame(file2, remove, False)
data_3 = getDataFrame(file3, remove, False)
data_4 = getDataFrame(file4, remove, False)

#man_data_01 = data_01[data_01['pmt'].isin(manualChs)]
#man_data_0 = data_0[data_0['pmt'].isin(manualChs)]
#plt.scatter(man_data_01['pmt'],man_data_01['q'],marker="s", label="Start (10263)")
#plt.scatter(man_data_0['pmt'],man_data_0['q'],marker="s", label="Start (10313)")

#plt.scatter(data_01['pmt'],data_01['q'],marker="s", label="Start (10263)")
plt.scatter(data_0['pmt'],data_0['q'],marker="s", label="Nominal (10890)")
plt.scatter(data_1['pmt'],data_1['q'],marker="s", label="+30V (10893)")
plt.scatter(data_2['pmt'],data_2['q'],marker="s", label="-30V (10894)")
plt.scatter(data_3['pmt'],data_3['q'],marker="s", label="-50V (10895)")
plt.scatter(data_4['pmt'],data_4['q'],marker="s", label="-100V (10896)")

plt.axhline(y=0.45, color="red", linestyle='dotted')
plt.legend()
plt.xlabel("PMT channel ID")
plt.ylabel("Gains [10^7 electrons]")
plt.grid()
plt.xlim((0, 180))
plt.savefig("equalization_runs.png",dpi=100)


In [ ]:
file01 = srcpath + "equalization_run10263_1691610710.csv" #nominal 
file0 = srcpath + "equalization_run10313_1692203859.csv" #nominal 

remove = offPMTs + EASTonly

data_01 = getDataFrame(file01, remove, False)
data_0 = getDataFrame(file0, remove, False)

#man_data_01 = data_01[data_01['pmt'].isin(manualChs)]
#man_data_0 = data_0[data_0['pmt'].isin(manualChs)]
#plt.scatter(man_data_01['pmt'],man_data_01['q'],marker="s", label="Start (10263)")
#plt.scatter(man_data_0['pmt'],man_data_0['q'],marker="s", label="Start (10313)")

plt.scatter(data_01['pmt'],data_01['q'],marker="s", label="Start (10263)")
plt.scatter(data_0['pmt'],data_0['q'],marker="s", label="Start (10313)")

plt.axhline(y=0.45, color="red", linestyle='dotted')
plt.legend()
plt.xlabel("PMT channel ID")
plt.ylabel("Gains [10^7 electrons]")
plt.grid()
plt.xlim((180.0, 359))
plt.savefig("manual_fix_highG.png",dpi=100)

In [ ]:
file1 = srcpath +"backgroundphotons_run10040_1688176609.csv"
file2 = srcpath +"backgroundphotons_run10058_1688566398.csv"
file3 = srcpath +"newicaruscode_run10058_1688574998.csv"
file4 = srcpath +"backgroundphotons_run10058_1688566368.csv"

data1 = getDataFrame(file1, offPMTs, False)
data2 = getDataFrame(file2, offPMTs, False)
data3 = getDataFrame(file3, offPMTs, False)
data4 = getDataFrame(file4, offPMTs, False)

In [ ]:
#plt.scatter(data1['pmt'],data1['q'],marker="o", label="10040 - new ")
plt.scatter(data2['pmt'],data2['q'],marker="o", label="10040 - old")
plt.scatter(data3['pmt'],data3['q'],marker="o", alpha=0.5, label="10058")
plt.scatter(data4['pmt'],data3['q'],marker="o", alpha=0.5, label="10097")

plt.axhline(y=0.45, color="red", linestyle='dotted')
plt.legend()
plt.xlabel("PMT channel ID")
plt.ylabel("Gains [10^7 electrons]")
plt.grid()
plt.xlim((0.0, 359))
plt.ylim((0.3, 0.5))

In [ ]:
#fitOutput1  = fitGainsDistribution( data1, BINSIZE=0.005,RMIN=0.0,RMAX=1.0, p0=(0.4, 0.04) )
fitOutput2  = fitGainsDistribution( data2, BINSIZE=0.005,RMIN=0.0,RMAX=1.0, p0=(0.4, 0.04) )
fitOutput3  = fitGainsDistribution( data3, BINSIZE=0.005,RMIN=0.0,RMAX=1.0, p0=(0.4, 0.04) )
fitOutput4  = fitGainsDistribution( data4, BINSIZE=0.005,RMIN=0.0,RMAX=1.0, p0=(0.4, 0.04) )

fig = plt.figure(figsize=(10, 5))
plt.xlim((0.3, 0.6))
#plt = makeplot(timestamp, figpath+"gain_1hist_"+str(run)+".pdf", *fitOutput1 )
plt = makeplot(timestamp, figpath+"gain_2hist_"+str(run)+".pdf", *fitOutput2 )
plt = makeplot(timestamp, figpath+"gain_3hist_"+str(run)+".pdf", *fitOutput3 )
plt = makeplot(timestamp, figpath+"gain_4hist_"+str(run)+".pdf", *fitOutput4 )

plt.savefig("runs_diff_files.png", dpi=900)